## ✫paraphrase-multilingual-MiniLM-L12-v2を使用して文章のベクトル化

### 1. 文章の本文をベクトル化するためにparaphrase-multilingual-MiniLM-L12-v2をロード

In [1]:
import sentence_transformers
from sentence_transformers import SentenceTransformer

In [2]:
# https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
# This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.
sbert = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
vector = sbert.encode('100回分のループを渡しているので, 1回のループで1%の進捗があります.')

In [3]:
vector.shape
# (384,)

(384,)

### 2. 文章の本文をリスト化

In [4]:
import os
# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

In [5]:
def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text

In [6]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)

### 3. Livedoorテキストの各本文をベクトル化して、以下の3つを作成する
- 文章ベクトル → 文章本文の辞書
- （文章ID, 文章ベクトル）を配列としたリスト
- 文章本文 → 文章ラベルの辞書

In [7]:
import numpy as np
livedoor_vec_to_text_dict = {}
livedoor_vec_list = []
livedoor_text_to_label_dict = {}

In [8]:
from tqdm import tqdm 

for text_item, label_item in tqdm(zip(list_text, list_label)):
    vector = sbert.encode(text_item)
    n = np.array(vector)
    n = tuple(n)
    livedoor_vec_to_text_dict[n] = text_item
    livedoor_vec_list.append((len(livedoor_vec_list), n))
    livedoor_text_to_label_dict[text_item] = label_item

7376it [01:17, 95.13it/s] 


In [9]:
print(len(list_text))
print(len(livedoor_vec_to_text_dict))
print(len(livedoor_vec_list))
print(len(livedoor_text_to_label_dict))
# 7376
# 7365 → 同じものがあるのか？
# 7376
# 7368 → 同じものがあるのか？

7376
7348
7376
7368


In [10]:
# 作成した辞書（文章ベクトル→文章本文）の検証
vector = sbert.encode(list_text[10])
n = np.array(vector)
n = tuple(n)
livedoor_vec_to_text_dict[n]

'イタリアのプレミアム・カジュアル・ブランド「DIESEL（ディーゼル）」は2月27日、28日の2日間、「0円」でTシャツを販売するゲリラストア「BE STUPID STORE」を原宿に限定オープンする。DIESELは、1991年から、斬新なテーマと強烈なビジュアルでメッセージ性の強い広告キャンペーンを展開している。2010年春夏シーズンのスローガンは「BE STUPID」。DIESELのスピリットであるポジティブな反骨精神が映し出されている言葉だ。「自分らしく生きること。失敗を恐れず、何度でも挑戦すること。そこにチャンスはやってくる」。ブランドのアイデンティティであるチャレンジ精神を伝えるために、DIESELは数々のフレーズとともに、このキャンペーンをさまざまな場面で展開していく。大人はNOと言う。僕らはYESと言う。大人は頭で考える。僕らは心で動く。大人には知恵がある。僕らには度胸がある。大人にはプランがある。僕らにはストーリーがある。僕らにしか分からない素晴らしさがある。BE STUPID.「BE STUPID STORE」では、このチャレンジ精神に賛同する人に、枚数限定のTシャツを0円で提供する。もし当日にTシャツを手に入れることが出来なくても、DIESELの公式サイトで、会員限定の特別待ち受け画像をダウンロードし、通常店舗で提示すれば、Tシャツをゲットすることが可能だ。■関連リンク・DIESEL - 公式サイト■店舗情報BE STUPID STORE東京都渋谷区神宮前4-31-16営業時間：12:00〜19:00※2月27日(土)、28日(日) 2日間限り'

In [11]:
# 10番目の文章とラベルを確認
print(list_text[10])
print(list_label[10])

イタリアのプレミアム・カジュアル・ブランド「DIESEL（ディーゼル）」は2月27日、28日の2日間、「0円」でTシャツを販売するゲリラストア「BE STUPID STORE」を原宿に限定オープンする。DIESELは、1991年から、斬新なテーマと強烈なビジュアルでメッセージ性の強い広告キャンペーンを展開している。2010年春夏シーズンのスローガンは「BE STUPID」。DIESELのスピリットであるポジティブな反骨精神が映し出されている言葉だ。「自分らしく生きること。失敗を恐れず、何度でも挑戦すること。そこにチャンスはやってくる」。ブランドのアイデンティティであるチャレンジ精神を伝えるために、DIESELは数々のフレーズとともに、このキャンペーンをさまざまな場面で展開していく。大人はNOと言う。僕らはYESと言う。大人は頭で考える。僕らは心で動く。大人には知恵がある。僕らには度胸がある。大人にはプランがある。僕らにはストーリーがある。僕らにしか分からない素晴らしさがある。BE STUPID.「BE STUPID STORE」では、このチャレンジ精神に賛同する人に、枚数限定のTシャツを0円で提供する。もし当日にTシャツを手に入れることが出来なくても、DIESELの公式サイトで、会員限定の特別待ち受け画像をダウンロードし、通常店舗で提示すれば、Tシャツをゲットすることが可能だ。■関連リンク・DIESEL - 公式サイト■店舗情報BE STUPID STORE東京都渋谷区神宮前4-31-16営業時間：12:00〜19:00※2月27日(土)、28日(日) 2日間限り
livedoor-homme


In [12]:
# 作成した辞書（文章ベクトル→文章本文）の検証
vector = sbert.encode(list_text[1015])
n = np.array(vector)
n = tuple(n)
livedoor_vec_to_text_dict[n]

'スペイン時間の19日、バレンシアに所属するアルゼンチン人MFエベル・バネガがとんだアクシデントに見舞われた。練習を終え、ガソリンスタンドに立ち寄ったバネガは、車から降りる際にサイドブレーキをかけ忘れた。よほどあわてたのか、バネガは進行する車を大事な商売道具である足で止めようとしたのだ。当然、車の圧力が止まるわけもなく、バネガは腓骨と脛骨を骨折する重傷。20日にも手術が行なわれる予定で、全治までは6カ月かかるとみられる。うっかりしたミスが、今季絶望の悲劇を招いてしまった。バレンシアの選手には、過去にもこうした「うっかり」ミスで重傷を負ったケースがあった。2002年5月、日韓W杯直前にスペイン代表GKホセサンティアゴ・カニサレスが、オーデコロンの瓶を誤って足に落とし、右足の腱(けん)を断裂。正GKとして臨むはずのW杯を、泣く泣く欠場するハメになった。バネガのケースはカニサレスよりも深刻であり、一歩間違えれば生命の危機もあった。「骨折で済んだ」こともあり、ネット上のファンからは「なぜ足で止めようと」「どこの三流コントだよ」「なんという逸材」「骨折界のファンタジスタやで」「トラップも満足に出来ねえのか」といった軽い反応が多く見られた。'

In [13]:
# 15番目の文章とラベルを確認
print(list_text[1015])
print(list_label[1015])

スペイン時間の19日、バレンシアに所属するアルゼンチン人MFエベル・バネガがとんだアクシデントに見舞われた。練習を終え、ガソリンスタンドに立ち寄ったバネガは、車から降りる際にサイドブレーキをかけ忘れた。よほどあわてたのか、バネガは進行する車を大事な商売道具である足で止めようとしたのだ。当然、車の圧力が止まるわけもなく、バネガは腓骨と脛骨を骨折する重傷。20日にも手術が行なわれる予定で、全治までは6カ月かかるとみられる。うっかりしたミスが、今季絶望の悲劇を招いてしまった。バレンシアの選手には、過去にもこうした「うっかり」ミスで重傷を負ったケースがあった。2002年5月、日韓W杯直前にスペイン代表GKホセサンティアゴ・カニサレスが、オーデコロンの瓶を誤って足に落とし、右足の腱(けん)を断裂。正GKとして臨むはずのW杯を、泣く泣く欠場するハメになった。バネガのケースはカニサレスよりも深刻であり、一歩間違えれば生命の危機もあった。「骨折で済んだ」こともあり、ネット上のファンからは「なぜ足で止めようと」「どこの三流コントだよ」「なんという逸材」「骨折界のファンタジスタやで」「トラップも満足に出来ねえのか」といった軽い反応が多く見られた。
sports-watch


### 4. 作成した辞書の永続化

In [14]:
import pickle
with open("livedoor_vec_to_text_dict.pkl","wb") as f:
    pickle.dump(livedoor_vec_to_text_dict, f)

with open("livedoor_text_to_label_dict.pkl","wb") as f:
    pickle.dump(livedoor_text_to_label_dict, f)   

### 5. ベクトルのリストの永続化

In [15]:
import pickle
with open("livedoor_vec_list.pkl","wb") as f:
    pickle.dump(livedoor_vec_list, f)